<a href="https://colab.research.google.com/github/moveformyfuture/KRX-Big-Data-Competition/blob/main/Document_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**한국어 문장요약 블로그 **

참고 : https://lovit.github.io/nlp/2019/05/01/krwordrank_sentence/

In [1]:
# 패키지 설치
!pip install krwordrank
!pip install soynlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 416 kB 5.1 MB/s 


In [ ]:
# 라이브러리 불러오기
import os
from pickle import FALSE
import re
import pandas as pd 
import numpy as np
import math
import pprint 
import datetime
from sklearn.model_selection import train_test_split
from functools import reduce
from typing import Text
from krwordrank.word import KRWordRank
from scipy.sparse import csr_matrix
from soynlp.tokenizer import MaxScoreTokenizer

# 크롤링으로 수집한 데이터를 전처리하여 필요한 필드를 추가하는 데이터
RAW_DATA_DIR = "..//data//orgin_data//"
PROCESSED_DATA_DIR = "..//news_article//data//processed//"
rawdata = "extract_data.xlsx"

class TextMing :
    def remove_unicode(input_str):
        # 유니코드 :  https://gist.github.com/Pusnow/aa865fa21f9557fa58d691a8b79f8a6d
        import unicodedata
        try:
            input_str = str(input_str)
            text = unicode(input_str, 'utf-8')
            print(text)
            
        except (TypeError, NameError): # unicode is a default on python 3 
            pass
        

        text = unicodedata.normalize('NFKC', input_str)
        text = text.encode('utf-8','ignore')
        text = text.decode("utf-8")
            
        return text

    def clean_ko_sentences(sentences):
        sentence = TextMing.remove_unicode(sentences)
        # 정규 표현식 필터
        RE_FILTER = re.compile(r"[#▲,<>!?\"'·:;=~()]") # [] 안에 제거할 정규표현식을 추가한다
        # 특수기호 제거
        sentences = re.sub(RE_FILTER, "", sentence)
        sentences = re.sub(r"[^A-Za-z0-9가-힣?!,¿]+", " ",sentences) 
        sentences = re.sub(r"\s"," ", sentences)# \n도 공백으로 대체해줌
        sentences = sentences.strip()  
        
        return str(sentences)


In [ ]:

def Morph_ko_document(sentence_List):
    # KoNLPy 형태소분석기 설정 '
    from konlpy.tag import Hannanum
    hannanum = Hannanum()

    nouns = []

    for sentence in sentence_List:
        sentence = TextMing.clean_ko_documents(sentence)
        # print(sentence)
        nouns_list = hannanum.nouns(str(sentence))
        nouns.append(nouns_list)
        # nouns_list =  TextMing.clean_ko_words(nouns_list)

    return nouns 


def extract(self, docs, beta=0.85, max_iter=10, num_keywords=-1,
    num_rset=-1, vocabulary=None, bias=None, rset=None):

    rank, graph = self.train(docs, beta, max_iter, vocabulary, bias)

    lset = {self.int2token(idx)[0]:r for idx, r in rank.items() if self.int2token(idx)[1] == 'L'}
    if not rset:
        rset = {self.int2token(idx)[0]:r for idx, r in rank.items() if self.int2token(idx)[1] == 'R'}

    if num_rset > 0:
        rset = {token:r for token, r in sorted(rset.items(), key=lambda x:-x[1])[:num_rset]}

    keywords = self._select_keywords(lset, rset)
    keywords = self._filter_compounds(keywords)
    keywords = self._filter_subtokens(keywords)

    if num_keywords > 0:
        keywords = {token:r for token, r in sorted(keywords.items(), key=lambda x:-x[1])[:num_keywords]}
    
    else : 
        pass

    return keywords, rank, graph


def make_vocab_score(keywords, scaling=None):
    if scaling is None:
        scaling = lambda x:math.sqrt(x)
    return {word:scaling(rank) for word, rank in keywords.items()}


def vectorize(sents, vocab_to_idx, tokenize):
    from scipy.sparse import csr_matrix
    from soynlp.tokenizer import MaxScoreTokenizer
    
    rows, cols, data = [], [], []
    for i, sent in enumerate(sents):
        terms = set(tokenize(sent))
        for term in terms:
            j = vocab_to_idx.get(term, -1)
            if j == -1:
                continue
            rows.append(i)
            cols.append(j)
            data.append(1)

    n_docs = len(sents)
    n_terms = len(vocab_to_idx)
    return csr_matrix((data, (rows, cols)), shape=(n_docs, n_terms))


def select(x, keyvec, texts, initial_penalty, topk=10):
    from sklearn.metrics import pairwise_distances

    dist = pairwise_distances(x, keyvec, metric='cosine').reshape(-1)
    dist = dist + initial_penalty

    idxs = []
    for _ in range(topk):
        idx = dist.argmin()
        idxs.append(idx)
        dist[idx] += 2 # maximum distance of cosine is 2
        idx_all_distance = pairwise_distances(
            x, x[idx].reshape(1,-1), metric='cosine').reshape(-1)
        penalty = np.zeros(idx_all_distance.shape[0])
        penalty[np.where(idx_all_distance < diversity)[0]] = 2
        dist += penalty
    return [texts[idx] for idx in idxs]

In [ ]:
from krwordrank.sentence import summarize_with_sentences

penalty = lambda x:0 if (25 <= len(x) <= 80) else 1
stopwords = {'영화', '관람객', '너무', '정말', '진짜'}
texts  = []


keywords, sents = summarize_with_sentences(
      texts,
      penalty=penalty,
      stopwords = stopwords,
      diversity=0.7,
      num_keywords=100,
      num_keysents=10,
      scaling=lambda x:1,
      verbose=False,)

In [ ]:
#핵심 단어 추출

from krwordrank.word import summarize_with_keywords

keywords = summarize_with_keywords(texts, min_count=5, max_length=10,
    beta=0.85, max_iter=10, stopwords=stopwords, verbose=True)
keywords = summarize_with_keywords(texts)